# **Faster R-CNN**

Postup vytvorenia modelu Faster R-CNN v tomto notebooku je založený na <a href="https://colab.research.google.com/drive/1ZuGY8s72SFlyiRKMfm_xKPwd_G4L-Kbd?usp=sharing#scrollTo=3B7uylHoP4FW">originálnom notebooku.</a>

### Príprava prostredia
Inštalácia knižnice Detectron2 a import potrebných knižníc

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import ColorMode

import torch, torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from PIL import Image
import cv2
import numpy as np
import IPython
import json
import os
import json
import csv
import time
#import random
from pathlib import Path

### Príprava dát
Konverzia YOLO anotácií na COCO formát

In [ ]:
def create_data_pairs(input_path, detectron_img_path, detectron_annot_path, dir_type = 'train'):

    img_paths = Path(input_path + dir_type + '/images/').glob('*.jpg')

    pairs = []
    for img_path in img_paths:

        file_name_tmp = str(img_path).split('/')[-1].split('.')
        file_name_tmp.pop(-1)
        file_name = '.'.join((file_name_tmp))

        label_path = Path(input_path + dir_type + '/labels/' + file_name + '.txt')

        if label_path.is_file():

            line_img = detectron_img_path + dir_type+'/images/'+ file_name + '.jpg'
            line_annot = detectron_annot_path+dir_type+'/labels/' + file_name + '.txt'
            pairs.append([line_img, line_annot])

    return pairs

In [ ]:
input_path = '../dataset/'
detectron_img_path = '../dataset/' 
detectron_annot_path = '../dataset/'

In [ ]:
train = create_data_pairs(input_path, detectron_img_path, detectron_annot_path, 'train')
val = create_data_pairs(input_path, detectron_img_path, detectron_annot_path, 'val')

In [ ]:
def create_coco_format(data_pairs):

    data_list = []

    for i, path in enumerate(data_pairs):

        filename = path[0]

        img_h, img_w = cv2.imread(filename).shape[:2]

        img_item = {}
        img_item['file_name'] = filename
        img_item['image_id'] = i
        img_item['height']= img_h
        img_item['width']= img_w

        print(str(i), filename)


        annotations = []
        with open(path[1]) as annot_file:
            lines = annot_file.readlines()
            for line in lines:
                if line[-1]=="\n":
                    box = line[:-1].split(' ')
                else:
                    box = line.split(' ')

                class_id = box[0]
                x_c = float(box[1])
                y_c = float(box[2])
                width = float(box[3])
                height = float(box[4])

                x1 = (x_c - (width/2)) * img_w
                y1 = (y_c - (height/2)) * img_h
                x2 = (x_c + (width/2)) * img_w
                y2 = (y_c + (height/2)) * img_h

                annotation = {
                    "bbox": list(map(float,[x1, y1, x2, y2])),
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "category_id": int(class_id),
                    "iscrowd": 0
                }
                annotations.append(annotation)
            img_item["annotations"] = annotations
        data_list.append(img_item)
    return data_list

In [ ]:
train_list = create_coco_format(train)
val_list = create_coco_format(val)

Nastavnie metadát

In [ ]:
for catalog_name, file_annots in [("train", train_list), ("val", val_list)]:
    DatasetCatalog.register(catalog_name, lambda file_annots = file_annots: file_annots)
    MetadataCatalog.get(catalog_name).set(thing_classes=['TLE'])

In [ ]:
metadata = MetadataCatalog.get("train")
MetadataCatalog.get("val")

### Trénovanie modelu

Nastavenie počtu iterácií a konfigurácie pre tréning

In [ ]:
batch_size = 8
max_iter = (int(len(train_list)/batch_size)) * 100
print(max_iter)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.DEVICE = 'cuda' # cpu
cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"
cfg.SOLVER.IMS_PER_BATCH = batch_size
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = max_iter
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 # 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get("train").thing_classes)
cfg.SOLVER.STEPS = (20500, )

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

import time as t
s1 = t.time()
try:
    trainer.train()
except:
    None
s2 = t.time()
print(s2-s1)

### Načítanie modelu

Načítanie modelu, počtu tried pre klasifikáciu a confidence threshold.

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.DEVICE = 'cuda' # cpu
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = "output/model_final.pth"   # cesta ku natrénovanému modelu
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.98   # skóre dôveryhodnosti (confidence threshold)
predictor = DefaultPredictor(cfg)

### Vizualizácia predikcií

Vykonanie vizualizácie na všetky obrázky v testovacej množine a následné uloženie do súborov:

*inferenced*:
>TP - súbor pre obrázky s detegovanými objektmi
 
>TN - súbor pre obrázky bez detegovaných objektov

In [ ]:
import os
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode
from PIL import Image

input_folder = "../dataset/test/images"
output_folder = "inferenced"
output_folder_tp = os.path.join(output_folder, "TP")
output_folder_tn = os.path.join(output_folder, "TN")

os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_tp, exist_ok=True)
os.makedirs(output_folder_tn, exist_ok=True)

predictor = DefaultPredictor(cfg)

for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
       
        im = cv2.imread(os.path.join(input_folder, filename))

        outputs = predictor(im)

        img = im.copy()
        annotated_img = img.copy()

        boxes = outputs["instances"].to("cpu").pred_boxes.tensor.numpy()
        class_ids = outputs["instances"].to("cpu").pred_classes.numpy()

        for box, class_id in zip(boxes, class_ids):
            box = [int(coord) for coord in box]
            class_name = "TLE"
            annotated_img = cv2.rectangle(np.array(img), (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)  # Green
     
        if len(boxes) > 0:
            output_path = os.path.join(output_folder_tp, filename)
        else:
            output_path = os.path.join(output_folder_tn, filename)

        Image.fromarray(annotated_img).save(output_path)